# Fine-tune meta-llama/Llama-3.2-1B-Instruct using mlx framework and llm-datasets

In [3]:
%pip install -r requirements.txt

  Using cached mlx_lm-0.20.5-py3-none-any.whl.metadata (9.2 kB)
  Using cached datasets-3.2.0-py3-none-any.whl.metadata (20 kB)
Using cached mlx_lm-0.20.5-py3-none-any.whl (133 kB)
Using cached datasets-3.2.0-py3-none-any.whl (480 kB)
Note: you may need to restart the kernel to use updated packages.


In [2]:
# set model name etc.

MODEL_NAME = "meta-llama/Llama-3.2-1B-Instruct"
MODEL_SHORT_NAME = MODEL_NAME.split('/')[-1]
SUFFIX = "FinGreyLit-MLX"
#SLICE = 1
print(MODEL_NAME)
print(MODEL_SHORT_NAME)

meta-llama/Llama-3.2-1B-Instruct
Llama-3.2-1B-Instruct


In [4]:
# Load and prepare fine-tuning dataset

import json
import glob
import random

random.seed(42)  # for deterministic sampling of test set

train_files = glob.glob("../../llm-dataset/*-train.jsonl") + glob.glob("../../llm-dataset/norrep.jsonl")
test_files = glob.glob("../../llm-dataset/*-test.jsonl")

EVAL_SIZE = 6  # how many documents to evaluate (i.e. calculate loss) on during fine-tuning
SYSTEM_PROMPT = "You are a skilled librarian specialized in meticulous cataloguing of digital documents."
INSTRUCTION = "Extract metadata from this document. Return as JSON."

def preprocess_sample(sample):
    output = json.dumps(sample["ground_truth"])
    input_ = json.dumps(sample["content"])
    # ShareGPT format
    conversations = [
        {'from': 'system', 'value': SYSTEM_PROMPT},
        {'from': 'user', 'value': INSTRUCTION + "\n\n" + input_},
        {'from': 'gpt', 'value': output}
    ]
    return {"conversations": conversations}

def dataset_to_records(files):
    records = []
    for filename in files:
        with open(filename) as infile:
            for line in infile:
                sample = json.loads(line)
                records.append(preprocess_sample(sample))
    return records

def write_jsonl(records, filename):
    with open(filename, "w") as outfile:
        for record in records:
            json.dump(record, outfile)
            outfile.write("\n")

train_recs = dataset_to_records(train_files)
random.shuffle(train_recs)
write_jsonl(train_recs, "mlx-train.jsonl")
print(f"Wrote {len(train_recs)} train records")

test_recs = dataset_to_records(test_files)
write_jsonl(test_recs, "mlx-test.jsonl")
print(f"Wrote {len(test_recs)} test records")

eval_recs = random.sample(test_recs, EVAL_SIZE)
write_jsonl(eval_recs, "mlx-eval.jsonl")
print(f"Wrote {len(eval_recs)} eval records")

Wrote 64 train records
Wrote 15 test records
Wrote 6 eval records


In [ ]:
# Load and finetune LLM
from mlx_lm import generate,load
from huggingface_hub import login

import os
from dotenv import load_dotenv